# Artificial Neural Networks and Deep Learning---## Homework 1: Minimal Working ExampleTo make your first submission, follow these steps:1. Create a folder named `[2024-2025] AN2DL/Homework 1` in your Google Drive.2. Upload the `training_set.npz` file to this folder.3. Upload the Jupyter notebook `Homework 1 - Minimal Working Example.ipynb`.4. Load and process the data.5. Implement and train your model.6. Submit the generated `.zip` file to Codabench.

## 🌐 Connect Colab to Google Drive

In [ ]:
!pip install tensorflow==2.17.0 keras==3.4.1 tensorflow-decision-forests==1.10.0 tensorflow-text==2.17.0 tf-keras==2.17.0 google keras_cv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 77.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/[2024-2025] AN2DL/Homework 1

## ⚙️ Import Libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import keras_cv

import numpy as np
import tensorflow as tf
import keras_cv
from tensorflow import keras as tfk
from tensorflow.keras import layers, models, applications
from tensorflow.keras.callbacks import LambdaCallback
import random
import keras
from keras.saving import register_keras_serializable

np.random.seed(42)
tf.random.set_seed(42);

## ⏳ Load the Data

In [ ]:
# Carica il dataset
data = np.load('training_set.npz')
X_train = data['images']
y_train = data['labels']

"""# Carica il dataset
data2 = np.load('/kaggle/input/blood-cells/test_set.npz')
X_test2 = data['images']
y_test2 = data['labels']"""

# Converti y in one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=8)
#y_test2 = tf.keras.utils.to_categorical(y_test2, num_classes=8)

In [ ]:
import hashlib

def image_hash(image):
    image_bytes = image.tobytes()
    return hashlib.sha256(image_bytes).hexdigest()

unique_images = []
unique_labels = []
duplicate_positions = set()

seen_hashes = {}

for i in range(len(X_train)):
    img_hash = image_hash(X_train[i])
    if img_hash not in seen_hashes:
        if i not in duplicate_positions:
            unique_images.append(X_train[i])
            unique_labels.append(y_train[i])
        seen_hashes[img_hash] = i
    else:
        duplicate_positions.add(seen_hashes[img_hash])
        duplicate_positions.add(i)

X_train = [X_train[i] for i in range(len(X_train)) if i not in duplicate_positions]
y_train = [y_train[i] for i in range(len(y_train)) if i not in duplicate_positions]

X_train = np.array(X_train)
y_train = np.array(y_train)


In [ ]:
# split train in training and test set
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


In [ ]:
X_test[0]

## 🛠️ Train and Save the Model

In [ ]:
class MyModel:
    def __init__(self):
        """
        Inizializza lo stato interno del modello MobileNetV3Small pre-addestrato.
        """
        self.neural_network = self.create_model()

    def get_augmentation_layer(self):
        # Definizione dei layer di data augmentation
        return tf.keras.Sequential([
            # Rotazione casuale
            keras.layers.RandomRotation(0.5, fill_mode='reflect'),

            # Zoom casuale in altezza
            keras.layers.RandomZoom(height_factor=(-0.2, 0.7), fill_mode='nearest'),

            # Altri tipi di augmentazioni
            keras.layers.RandomZoom(height_factor=(0.0, 0.0), width_factor=(-0.3, 0.3), fill_mode='nearest'),
            keras.layers.RandomFlip(mode="horizontal"),
            keras.layers.RandomFlip(mode="vertical"),
            keras.layers.RandomTranslation(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)),

            keras.layers.RandomBrightness(0.3),
            keras_cv.layers.RandomHue(0.3, [0,255]),

            # Aggiunta di rumore gaussiano
            keras.layers.GaussianNoise(0.05),

            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),

            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),

            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),

            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),

            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),

            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
            keras_cv.layers.RandomCutout(0.075, 0.075, fill_mode="gaussian_noise"),
      ])


    def create_model(self):
        """
        Crea e restituisce un modello con MobileNetV3Small.
        """
        # Definisci i layer di data augmentation
        data_augmentation = self.get_augmentation_layer()

        # Utilizza una rete pre-addestrata
        model_pretrained = tfk.applications.ConvNeXtLarge(
            input_shape=(96, 96, 3),
            include_top=False,
            weights='imagenet',
            pooling='avg'  # Pooling globale per ridurre la dimensionalità
        )
        self.model_name_pretrained = 'convnext_large'

        print("number of layers:")
        print(len(model_pretrained.layers))

        # Costruisci il modello
        inputs = tfk.Input(shape=(96, 96, 3), name='input_layer')
        x = data_augmentation(inputs)
        x = model_pretrained(x)

        x = tfk.layers.Dense(512, activation='relu')(x)
        x = tfk.layers.Dropout(0.3)(x)
        x = tfk.layers.Dense(256, activation='relu')(x)
        x = tfk.layers.Dropout(0.3)(x)
        '''
        x = tfk.layers.Dense(128, activation='relu')(x)
        x = tfk.layers.Dropout(0.3)(x)'''
        outputs = tfk.layers.Dense(8, activation='softmax', name='output_layer')(x)

        model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

        return model

    def train_transfer_learning(self, X_train, y_train, X_test, y_test, epochs=10, batch_size=32):
        """
        Pre-addestra il modello con i layer congelati.
        """

        self.neural_network.get_layer(self.model_name_pretrained).trainable = False

        # Sblocca i layer convoluzionali dal layer `fine_tune_from` in poi
        for i, layer in enumerate(self.neural_network.get_layer(self.model_name_pretrained).layers):
            layer.trainable = False

        # Ricompila il modello (necessario dopo aver modificato i layer trainabili)
        self.neural_network.compile(
            loss=tfk.losses.CategoricalCrossentropy(),
            optimizer=tfk.optimizers.Lion(),
            metrics=['accuracy']
        )

        # Callback
        save_every_10 = LambdaCallback(
            on_epoch_end=lambda epoch, logs:
            self.neural_network.save(f'Large/model_epoch_{epoch + 1}.keras') if (epoch + 1) % 10 == 0 else None
        )
        early_stopping = tfk.callbacks.EarlyStopping(
            monitor='val_accuracy',
            mode='max',
            patience=8,
            restore_best_weights=True
        )

        # Riaddestra il modello
        history = self.neural_network.fit(
            x=X_train,
            y=y_train,
            batch_size=batch_size,
            epochs=epochs,
            shuffle=True,
            validation_data=(X_test, y_test),
            callbacks=[save_every_10, early_stopping]
        )

    def train_fine_tuning(self, X_train, y_train, X_test, y_test, epochs=10, batch_size=32, fine_tune_from=50):
        """
        Sblocca i layer selezionati e riaddestra il modello.
        """

        self.neural_network.get_layer(self.model_name_pretrained).trainable = True

        # Sblocca i layer convoluzionali dal layer `fine_tune_from` in poi
        for i, layer in enumerate(self.neural_network.get_layer(self.model_name_pretrained).layers):
            layer.trainable = False
            if i > fine_tune_from:
              if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.DepthwiseConv2D):
                  layer.trainable = True


        # Ricompila il modello con un learning rate più basso
        self.neural_network.compile(
            loss=tfk.losses.CategoricalCrossentropy(),
            optimizer=tfk.optimizers.Lion(learning_rate=1e-4),
            metrics=['accuracy']
        )

        # Callback
        save_every_10 = LambdaCallback(
            on_epoch_end=lambda epoch, logs:
            self.neural_network.save(f'Large/model_epoch_{epoch + 1}.keras') if (epoch + 1) % 10 == 0 else None
        )
        early_stopping = tfk.callbacks.EarlyStopping(
            monitor='val_accuracy',
            mode='max',
            patience=8,
            restore_best_weights=True
        )

        # Riaddestra il modello
        history = self.neural_network.fit(
            x=X_train,
            y=y_train,
            batch_size=batch_size,
            epochs=epochs,
            shuffle=True,
            validation_data=(X_test, y_test),
            callbacks=[save_every_10, early_stopping]
        )

    def test(self, X_test, y_test):
        """
        Valuta il modello sui dati di test X_test e le etichette y_test.
        """
        test_loss, test_acc = self.neural_network.evaluate(X_test, y_test)
        print(f'Test accuracy: {test_acc}')

    def save(self):
        """
        Salva il modello senza i layer di data augmentation.
        """
        self.neural_network.save('/gdrive/MyDrive/[2024-2025] AN2DL/Homework 1/Large/weights.keras')

    def predict(self, X):
        """
        Predice le etichette corrispondenti all'input X.
        """
        preds = self.neural_network.predict(X)
        preds = np.argmax(preds, axis=1)
        return preds



In [ ]:
model = MyModel()

In [ ]:
data_augmentation = model.get_augmentation_layer()

# Applica la rotazione casuale
rotated_image = data_augmentation(X_train[6001])
print(y_train[6001])

# Visualizza l'immagine originale e quella ruotata
plt.figure(figsize=(10, 5))

# Mostra l'immagine ruotata
plt.subplot(1, 2, 1)
plt.imshow(X_train[6001])
plt.title("Immagine Ruotata")
plt.axis('off')

# Mostra l'immagine ruotata
plt.subplot(1, 2, 2)
plt.imshow(rotated_image / 255.0)
plt.title("Immagine Ruotata")
plt.axis('off')

plt.show()

In [ ]:
#model.neural_network.save('weights2.keras')

In [ ]:
#X_test_augmented = model.get_augmentation_layer()(X_test)

In [ ]:
model.train_transfer_learning(X_train, y_train, X_test, y_test, 20, 512)
#model.train_transfer_learning(X_train, y_train, X_test, y_test, 1, 512)

In [ ]:
model.neural_network.save('/gdrive/My Drive/[2024-2025] AN2DL/Homework 1/Large/weights.keras')

In [ ]:
model.train_fine_tuning(X_train, y_train, X_test, y_test, 50, 512, 220)
#model.train_fine_tuning(X_train, y_train, X_test, y_test, 1, 512, 220)

In [ ]:
model.neural_network.save('/gdrive/My Drive/[2024-2025] AN2DL/Homework 1/Large/weights2.keras')

In [ ]:
#model.test(X_test2, y_test2)

In [ ]:
#preds = model.predict(X_test)

In [ ]:
"""import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
y_true = np.argmax(y_test, axis=1)
y_pred = preds
# Genera la confusion matrix
cm = confusion_matrix(y_true, y_pred)
# Visualizza la confusion matrix con le etichette
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
# Usa una mappa di colori blu
plt.title("Confusion Matrix")
plt.show()"""

In [ ]:
#model.neural_network.save('/gdrive/MyDrive/weights.keras')

## 📊 Prepare Your SubmissionTo prepare your submission, create a `.zip` file that includes all the necessary code to run your model. It **must** include a `model.py` file with the following class:```python# file: model.pyclass Model:    def __init__(self):        """Initialize the internal state of the model."""    def predict(self, X):        """Return a numpy array with the labels corresponding to the input X."""```The next cell shows an example implementation of the `model.py` file, which includes loading model weights from the `weights.keras` file and conducting predictions on provided input data. The `.zip` file is created and downloaded in the last notebook cell.❗ Feel free to modify the method implementations to better fit your specific requirements, but please ensure that the class name and method interfaces remain unchanged.

In [ ]:
%%writefile '/gdrive/My Drive/[2024-2025] AN2DL/Homework 1/Large/model.py'
class Model2:
  def __init__(self):        # Carica il modello senza compilazione
    self.neural_network = tfk.models.load_model('weights.keras')

  def test(self, X_test, y_test):
    test_loss, test_acc = self.neural_network.evaluate(X_test, y_test)
    print(f'Test accuracy: {test_acc}')

  def predict(self, X):
    preds = self.neural_network.predict(X)
    if len(preds.shape) == 2:
      preds = np.argmax(preds, axis=1)
      return preds

In [ ]:
#model2 = Model2()model2.test(X_test2, y_test2)

In [ ]:
%cd Large

from datetime import datetime
filename1 = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}_NOFN.zip'
filename2 = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}_FN.zip'

# Add files to the zip command if needed
!zip {filename1} model.py weights.keras
!zip {filename2} model.py weights2.keras

from google.colab import files
files.download(filename1)
files.download(filename2)